#### imports

In [97]:
import numpy as np
%matplotlib qt
from sympy import Matrix, latex
import pprint
import math
import time
import random
import itertools
import matplotlib.pyplot as plt
from qiskit_optimization.algorithms import MinimumEigenOptimizer, CplexOptimizer, GurobiOptimizer, CobylaOptimizer
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_optimization.algorithms import GurobiOptimizer
from qiskit_optimization import QuadraticProgram
from qiskit_aer import Aer, AerSimulator
from qiskit.primitives import StatevectorSampler, Sampler
from qiskit_algorithms.utils import algorithm_globals
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister, Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator, QiskitRuntimeService
from qiskit.circuit.library import RealAmplitudes
from scipy.optimize import minimize, OptimizeResult

#### matrix stuff

In [77]:
def rand_sp_int_mat(nv,seed,sp): 
    
    #### sp is the percentage of non-zero elements. Bigger number: more dense. 1: Fully dense, ~0: very sparse
    spp = str(int(sp*10))
    
    #### Picking the non-zero off diagonal entries
    rs1 = np.random.RandomState(seed)
    matrix = np.zeros((nv,nv))
    upp_diag = int((nv**2)/2 - (nv/2)) # number of elements in upper diagonal. Same as above
    ind_list = rs1.choice(upp_diag,int(sp*upp_diag), replace=False)
    
    randindex = list([])
    
    k = 0
    for item in itertools.combinations([x for x in range(nv)], r=2):       
        if k in ind_list:
            randindex.append(list(item))
        else:
            pass
        k = k +1
    
    # Setting the values the non-zero off diagonal entries
    for item in randindex:
        drawrand = rs1.randint(-5,5)
        while drawrand == 0: # While loop to prevent drawing extra zeros
            drawrand = rs1.randint(-5,5)
        matrix[item[0],item[1]] = drawrand
        matrix[item[1],item[0]] = drawrand
        
        
    # Picking the non-zero diagonal entries
    rs2 = np.random.RandomState(2*seed)
    ind_list = rs2.choice(nv,int(sp*nv), replace=False)
    
    # Setting the non-zero diagonal entries
    k=0
    for s in range(nv):
        if k in ind_list:
            drawrand = rs2.randint(-5,5)
            while drawrand == 0: # While loop to prevent drawing extra zeros
                drawrand = rs2.randint(-5,5)
            matrix[s][s] = drawrand
            
        else:
            pass
        k = k+1
   
    return matrix, "sii"+str(seed) + "." + str(spp)


def get_mat_sp(matrix): # Gets the "sp" value from the matrix.
    unique, counts = np.unique(matrix, return_counts=True)
    tt1 = dict(zip(unique, counts))
    try:
        mm1 = 1-(tt1[0]/np.sum(counts)) # ratio of zeros to total number of elements
    except KeyError:
        mm1 = 0
        
    return np.round(mm1,5)

def symmetric_qubo_to_upper_triangular(matrix):
    n = len(matrix)
    upper_triangular_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(i, n):
            if i != j:
                upper_triangular_matrix[i, j] = 2 * matrix[i, j]
            else:
                upper_triangular_matrix[i, j] = matrix[i, j]

    return upper_triangular_matrix

def uppertriangularmatrix(matrix): 
    mat= [[matrix[i][j] for j in range(i, len(matrix))] for i in range(len(matrix))]
    return mat

def random_qubo_matrix(nc, seed, sparse):
    return(symmetric_qubo_to_upper_triangular(rand_sp_int_mat(nc, seed, sparse)[0]))


def blockmatrix(na, nr,seed):
    ## seed is a vector that helps us fix the sparsity of each block matrix
    ## na is the length of each block 
    ## number of variables nc is equal to the length of the matrix 
    ## so we create matrix of blocks 
    nc = 2**nr*na    
    Q=[]
    
    for i in range(2**nr):   
        
        a1 = symmetric_qubo_to_upper_triangular(rand_sp_int_mat(na,random.randint(0,2000),seed[i])[0])

        #zeros_column_left = ((matrix.shape[0], i*na))
        #zeros_column_right = np.zeros((matrix.shape[0], (nc-i-1)*na))

        # Columns of zeros to append
        zeros_column_left = np.zeros((na, i*na))
        zeros_column_right = np.zeros((na, (2**nr-i-1)*na))

            # Append the column of zeros on the left and right side of the matrix
        a1 = np.hstack((zeros_column_left, a1, zeros_column_right)).tolist()
        Q.append(a1)
    return np.concatenate(Q, axis=0)

## Generate your qubo matrix

In [78]:
Q= random_qubo_matrix(4, 18, 1)
Q

array([[ 3., -8., -6., -6.],
       [ 0., -4.,  6.,  6.],
       [ 0.,  0.,  4., -6.],
       [ 0.,  0.,  0.,  2.]])

In [79]:
Q=blockmatrix(4,1,[0.3,1])
pprint.pprint(blockmatrix(4,1,[0.3,1]))

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  6.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1., -6.,  6.,  2.],
       [ 0.,  0.,  0.,  0.,  0., -3., -6., -8.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -4.,  2.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.]])


## GUROBI solver

In [90]:
def gurobi(Q):
    import time

    # Create a QuadraticProgram object
    qp = QuadraticProgram()

    # Define the QUBO variables
    num_vars = len(Q)
    for i in range(num_vars):
        qp.binary_var(f'x_{i}')

    # Set the QUBO objective
    qp.minimize(linear=[0]*num_vars, quadratic=Q)

    problem = qp.prettyprint()
    
    # Create a GurobiOptimizer object
    optimizer = GurobiOptimizer()

    # Solve the QP problem
    result_guro_min = optimizer.solve(qp)  

    # Print the solution
    solution_min = result_guro_min.prettyprint()

    C_min=result_guro_min.fval

    qp_max = QuadraticProgram()

    Q_max=np.multiply(Q,-1)    

    for i in range(num_vars):
        qp_max.binary_var(f'x_{i+1000000}')
    qp_max.minimize(linear=[0]*num_vars, quadratic=Q_max)

    start_time = time.time()

    # Solve the QP problem
    result_guro_max = optimizer.solve(qp_max)

    # Print the solution
    solution_max = result_guro_max.prettyprint()

    C_max=-result_guro_max.fval
    
    end_time = time.time()
    execution_time = end_time - start_time 

    return C_min , C_max, execution_time
    #return solution_min , solution_max

## Solve your problem here using GUROBI

In [98]:
C_min , C_max, time = gurobi(Q)
C_min, C_max, time

(-23.0, 8.0, 0.002505064010620117)

# 
 * *
   
 *Solve your problem with 1 qubit = 1 variable*

 * *

## QAOA with local simulator (need to implement warm starting and recursive QAOA)

In [99]:
def QAOA_run(Q, depth, noise): 
    import time

# Create a QuadraticProgram object
    qp = QuadraticProgram()

    # Define the QUBO variables
    num_vars = len(Q)
    for i in range(num_vars):
        qp.binary_var(f'x_{i}')

    # Set the QUBO objective
    qp.minimize(linear=[0]*num_vars, quadratic=Q)

    problem = qp.prettyprint()

    start_time = time.time()
    
    meo = MinimumEigenOptimizer(QAOA(sampler=Sampler(), optimizer=COBYLA(maxiter=100), reps = depth))

    " for further details see : https://docs.quantum.ibm.com/api/qiskit/0.26/qiskit.algorithms.QAOA " 

    # Solve the problem
    result = meo.solve(qp)
    end_time = time.time()
        
    execution_time = end_time - start_time
    # Print the solution
    print(result)  # prints the optimal value and the optimal solution
    print(result.prettyprint())
    print(execution_time)

In [100]:
" be aware that it can take some time for nc >= 16 and that for nc >= 32, there is not enough on the simulators"

QAOA_run(Q, 1, False)

fval=-23.0, x_0=0.0, x_1=1.0, x_2=1.0, x_3=0.0, x_4=0.0, x_5=1.0, x_6=1.0, x_7=1.0, status=SUCCESS
objective function value: -23.0
variable values: x_0=0.0, x_1=1.0, x_2=1.0, x_3=0.0, x_4=0.0, x_5=1.0, x_6=1.0, x_7=1.0
status: SUCCESS
0.7107300758361816


# 
 * *
   
 *Solve your problems with more variables*

 * *

## Angel Q algorithm : minimal encoding 

#### Minimal encoding solver

In [94]:
def minimal_encoding(Q, shots, depth, noise, optimization_level, num_solutions, max_iter, tol): 
    ## nc is the number of classical variables (it must be a power of 2 for minimal encoding)
    ## sparse value is the sparsity of the qubo matrix, seed is any random integer you want for the cration of the QUBO matrix 
    ## shots, depth are for the sampling process
    ## noise is True or False depending if you want a noise simulation. optimization_level is 0, 1 or 2 depending on the noise mitigation level you want for your 
    # num_solutions define the number of classical solution samples
    # max_iter defines the max number of iterations of the optimization run 
    import time

    
    na=1
    nc = len(Q)
    Q_upper_triangular=uppertriangularmatrix(Q)
    nr=int(math.log2(nc))

    nq=nr+na

    ## creating the quantum circuit 


    #initialize the quantum circuit
    c_me=QuantumCircuit(nq)
    
    for i in range(nq):
         c_me.h(i)          # put all qubits in state superposition using Hadamard gate
     
    theta_me=[0 for i in range(nq)] +[np.pi for i in range(nq*(depth))]  
    
    
    #To cancel first rotation of the ansatz 
    for i in range(nq):
        c_me.ry(-theta_me[i],i)
    
    #ansatz
    ansatz= RealAmplitudes(nq, entanglement='pairwise',reps=depth,parameter_prefix='theta_me', insert_barriers=True)
    
    c_me.compose(ansatz, inplace=True)
    c_me.measure_all()

    #initialization of the sampler
    if noise == False : 
        sampler = StatevectorSampler()
        isa_c_me= c_me
    else : 
        service = QiskitRuntimeService()
 
        # Specify a system to use for the noise model
        real_backend = service.backend("ibm_sherbrooke")
        aer = AerSimulator.from_backend(real_backend)
        
        # Run the sampler job locally using AerSimulator.
        pm = generate_preset_pass_manager(backend=aer, optimization_level=optimization_level)  ## 

        isa_c_me = pm.run(c_me)
        #qc_noise2.draw("mpl"), isa_qc2.draw("mpl")
        sampler = SamplerV2(backend=aer)

    def run_job(theta_me):
        job = sampler.run([(isa_c_me, theta_me)],shots=shots)
        job_result = job.result()
        return job_result


        ## cost function computation 

    def faster(p_vector,upper_triangular_qubo_matrix):
        ## first, we construct the C matrix 
        C= [[0 for j in range(i, nc)] for i in range(nc)]
        # Step 1: Calculate the diagonal coefficients of C
            
        # Iterate over the indices of matrix qubo_matrix
        for i in range(nc):
            # Check if the element is on the diagonal and non-zero
                # Assign 1 to diagonal elements of C
            C[i][0] = p_vector[i]
        
        # Step 2: Compute the non-diagonal coefficients of C
        # Iterate over the indices of matrix qubo_matrix
        for i in range(nc):
            for j in range(1, nc-i):
                # Check if the corresponding diagonal element of A is non-zero
                if upper_triangular_qubo_matrix[i][j] != 0:
                    # Assign to non-diagonal elements of C
                    C[i][j] = C[i][0]*C[j+i][0] 
        return C
    
    def cf_me_faster(qubo_matrix, theta):
        S=0
        A=faster(prob_vec(theta),Q_upper_triangular)
        for i in range(nc): 
            for j in range(0, nc-i):
                S+=qubo_matrix[i][j]*A[i][j]
        return S
    
    def cff_me_faster(theta):
            return cf_me_faster(Q_upper_triangular,theta)


    #create probability matrix and sampling at the same time
    def prob_vec(theta):
        p_vec = np.zeros((2**nr))
    
        job_result_me = run_job(theta)
        
        #sorted dictionary
        results_me={k: job_result_me[0].data.meas.get_counts()[k] for k in sorted(job_result_me[0].data.meas.get_counts())}
    
        sum_dict = {format(k, '0' + str(nr) + 'b') : 0 for k in range(2**nr)}
    
        for i in range(2**(nq)):
            
            binary_str = format(i, '0' + str(nq) + 'b')  # Convert integer to binary string
            
            register = binary_str[-nr:]
            
            if binary_str in results_me: 
                sum_dict[register] +=results_me[binary_str]
            
                if binary_str[0]=='1':
                            p_vec[int(register,2)] = results_me[binary_str]
     
        for i, (register, s) in enumerate(sum_dict.items()):
            if s != 0:
                p_vec[i] /= s
            else:                                    
                p_vec[i]=0.5              #safeguard in case register is not measured
    
        p_vec=p_vec.tolist() #results_me #if i want to double check the results
        return p_vec #,results_me
             
    rhobeg = 0.1*2*np.pi  # Radius of Cobyla linear approximation. Adjust this value based on your problem and experimentation if COBYLA struggles
                      # BUT make sure to check if you have enough shots and depth before touching this parameter
    

    def build_callback(callback_dict):
        """Return callback function that uses Estimator instance,
        and stores intermediate values into a dictionary.
    
        Parameters:
            ansatz (QuantumCircuit): Parameterized ansatz circuit
            hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
            estimator (EstimatorV2): Estimator primitive instance
            callback_dict (dict): Mutable dict for storing values
    
        Returns:
            Callable: Callback function object
        """
    
        def callback(current_vector):
            """Callback function storing previous solution vector,
            computing the intermediate cost value, and displaying number
            of completed iterations and average time per iteration.
    
            Values are stored in pre-defined 'callback_dict' dictionary.
    
            Parameters:
                current_vector (ndarray): Current vector of parameters
                                          returned by optimizer
            """
            # Keep track of the number of iterations
            callback_dict["iters"] += 1
            # Set the prev_vector to the latest one
            callback_dict["prev_vector"] = current_vector
            # Compute the value of the cost function at the current vector
            # This adds an additional function evaluation
            current_cost = cff_me_faster(current_vector)
            callback_dict["cost_history"].append(current_cost)
            # Print to screen on single line
            print(
                "Iters. done: {} [Current cost: {}]".format(callback_dict["iters"], current_cost),
                end="\r",
                flush=True,
            )
    
        return callback
    
    #List of sampling of classical solution 

    classical_solutions=[]
    
    ##creating callback dictionary
    
    callback_dict = {
        "prev_vector": None,
        "iters": 0,
        "cost_history": [],
    }
    callback = build_callback(callback_dict)
    
    # Execute the variational quantum circuit
    
    start_time = time.time()
    
    for _ in range(num_solutions):              # Execute quantum circuit and obtain outcomes
    
            
        result_cobyla_me_dp = minimize(cff_me_faster, [np.pi * np.random.uniform(0, 2) for i in range(nq*(depth + 1))],
                                   
                                       method="COBYLA",
                                   
                                       bounds=[(0, 2*np.pi) for _ in range(nq*(depth + 1))],
                                   
                                       callback=callback,
                                   
                                       options={'maxiter': max_iter, 'tol': tol, 'disp': True})
    
        
        classical_solutions.append(result_cobyla_me_dp.x)
        
    end_time = time.time()
    
    execution_time = end_time - start_time
    
    # Extract the cost history from the dictionary
    cost_history_normalized = [(cost-C_min)/(C_max-C_min) for cost in callback_dict['cost_history']]
    #cost_history_normalized = [cost for cost in callback_dict['cost_history']]
    
    # Plot the cost history
    plt.plot(cost_history_normalized)
    plt.xlabel('Iteration')
    plt.ylabel('Cost')
    plt.title(f'Cost History Normalized,  nc = {nc},   depth = {depth},   shots = {shots},   noise = {noise},   iter max = {max_iter}')
    plt.grid(True)
    plt.show()
        
        # ANSI escape code for bold text
    bold_text = "\033[1m"
        # ANSI escape code to reset text formatting
    reset_format = "\033[0m"

    print("")

    print("The solutions are : ", classical_solutions)
    #print("The normalised cost history is : ", cost_history_normalized)
    print("")

    solution_average_me = 0

    shots = 30000 #to get a precise solution 

    # Print the text in bold
    for i in range(num_solutions):
        #for j in range(17): 
            #print(bold_text + "The solution is", format(np.round((sample10(sample_to_vec(final_result_cobyla_me_dp[i]))[j])))
        print(bold_text + "The solution is", format(np.round((prob_vec(classical_solutions[i]))))
         + reset_format)
        #print(bold_text + "The solution value is", format(np.dot(np.dot(np.transpose(np.round((sample10(sample_to_vec(final_result_cobyla_me_dp[i]))[j]))),Q),np.round((sample10(sample_to_vec(final_result_cobyla_me_dp[i]))[j]))))
        print(bold_text + "The solution value is", format(np.dot(np.dot(np.transpose(np.round((prob_vec(classical_solutions[i])))),Q),np.round((prob_vec(classical_solutions[i])))))
         + reset_format) 
        print("")

        solution_average_me+=np.dot(np.dot(np.transpose(np.round((prob_vec(classical_solutions[i])))),Q),np.round((prob_vec(classical_solutions[i]))))
        print(f"""probability matrix value in percentage: {(np.floor(np.multiply(prob_vec(classical_solutions[i]),100)))}""")
    print(f"""Number of iterations per optimization : {callback_dict['iters']/num_solutions}""")
    print(f"""Number of second per iteration : {execution_time/callback_dict['iters'] } seconds""")
    print(f"""Number of second per optimization : {execution_time/num_solutions } seconds""")
    print(f"""Time (mins): {execution_time//60} mins and {execution_time%60} sec""")
    print(f"""Cost function value: {callback_dict['cost_history'][-1]}""")
    print(f"""Solution average: {solution_average_me/num_solutions}""")


#### Minimal encoding solver using SGD

In [95]:
def minimal_encoding_sgd(Q, shots, depth, noise, optimization_level, num_solutions, SGD): 
    ## nc is the number of classical variables (it must be a power of 2 for minimal encoding)
    ## sparse value is the sparsity of the qubo matrix, seed is any random integer you want for the cration of the QUBO matrix 
    ## shots, depth are for the sampling process
    ## noise is True or False depending if you want a noise simulation. optimization_level is 0, 1 or 2 depending on the noise mitigation level you want for your 
    # num_solutions define the number of classical solution samples
    # max_iter defines the max number of iterations of the optimization run 
    import time

    na=1
    nc = len(Q)
    Q_upper_triangular=uppertriangularmatrix(Q)
    nr=int(math.log2(nc))

    nq=nr+na

    ## creating the quantum circuit 


    #initialize the quantum circuit
    c_me=QuantumCircuit(nq)
    
    for i in range(nq):
         c_me.h(i)          # put all qubits in state superposition using Hadamard gate
     
    theta_me=[0 for i in range(nq)] +[np.pi for i in range(nq*(depth))]  
    
    
    #To cancel first rotation of the ansatz 
    for i in range(nq):
        c_me.ry(-theta_me[i],i)
    
    #ansatz
    ansatz= RealAmplitudes(nq, entanglement='pairwise',reps=depth,parameter_prefix='theta_me', insert_barriers=True)
    
    c_me.compose(ansatz, inplace=True)
    c_me.measure_all()

    #initialization of the sampler
    if noise == False : 
        sampler = StatevectorSampler()
        isa_c_me= c_me
    else : 
        service = QiskitRuntimeService()
 
        # Specify a system to use for the noise model
        real_backend = service.backend("ibm_sherbrooke")
        aer = AerSimulator.from_backend(real_backend)
        
        # Run the sampler job locally using AerSimulator.
        pm = generate_preset_pass_manager(backend=aer, optimization_level=optimization_level)

        isa_c_me = pm.run(c_me)
        #qc_noise2.draw("mpl"), isa_qc2.draw("mpl")
        sampler = SamplerV2(backend=aer)

    def run_job(theta):
        job = sampler.run([(isa_c_me, theta)],shots=shots)
        job_result = job.result()
        return job_result


        ## cost function computation 

    def faster(p_vector,upper_triangular_qubo_matrix):
        ## first, we construct the C matrix 
        C= [[0 for j in range(i, nc)] for i in range(nc)]
        # Step 1: Calculate the diagonal coefficients of C
            
        # Iterate over the indices of matrix qubo_matrix
        for i in range(nc):
            # Check if the element is on the diagonal and non-zero
                # Assign 1 to diagonal elements of C
            C[i][0] = p_vector[i]
        
        # Step 2: Compute the non-diagonal coefficients of C
        # Iterate over the indices of matrix qubo_matrix
        for i in range(nc):
            for j in range(1, nc-i):
                # Check if the corresponding diagonal element of A is non-zero
                if upper_triangular_qubo_matrix[i][j] != 0:
                    # Assign to non-diagonal elements of C
                    C[i][j] = C[i][0]*C[j+i][0] 
        return C
    
    def cf_me_faster(qubo_matrix, theta):
        S=0
        A=faster(prob_vec(theta),Q_upper_triangular)
        for i in range(nc): 
            for j in range(0, nc-i):
                S+=qubo_matrix[i][j]*A[i][j]
        return S
    
    def cff_me_faster(theta):
            return cf_me_faster(Q_upper_triangular,theta)


    def proj_list(sampling_result,na,nr):     ##take a sampling_result dictionary unsorted or sorted

        p_vec = np.zeros((2**nr))
        #sorted dictionary
        results_me={k: sampling_result[0].data.meas.get_counts()[k] for k in sorted(sampling_result[0].data.meas.get_counts())}
    
        sum_dict = {format(k, '0' + str(nr) + 'b') : 0 for k in range(2**nr)}
    
        for i in range(2**nq):
            
            binary_str = format(i, '0' + str(nq) + 'b')  # Convert integer to binary string
            
            register = binary_str[-nr:]
            
            if binary_str in results_me: 
                
                sum_dict[register] +=results_me[binary_str]
            
                if binary_str[0]=='1':
                            p_vec[int(register,2)] = results_me[binary_str]
     
        for i, (register, s) in enumerate(sum_dict.items()):
            if s != 0:
                p_vec[i] /= s
    
            else:                                    
                p_vec[i]=0.5
                sum_dict[register] +=1     #safeguard in case register is not measured
                
        
        reg_proj = [sum_dict[key]/shots for key in sum_dict]
        
        p_vec=p_vec.tolist() 
        
        proj_1 = [p_vec[i]* reg_proj[i//na] for i in range(nc) ]
        
        return np.array(reg_proj), np.array(proj_1)
        
    def probregister(job_result, na, nr):
        return proj_list(job_result,na,nr)[0]
    
    def proj_1(job_result, na, nr): 
        return proj_list(job_result,na,nr)[1]
    
    #create probability matrix and sampling at the same time
    def prob_vec(theta):
        p_vec = np.zeros((2**nr))
    
        job_result_me = run_job(theta)
        
        #sorted dictionary
        results_me={k: job_result_me[0].data.meas.get_counts()[k] for k in sorted(job_result_me[0].data.meas.get_counts())}
    
        sum_dict = {format(k, '0' + str(nr) + 'b') : 0 for k in range(2**nr)}
    
        for i in range(2**(nq)):
            
            binary_str = format(i, '0' + str(nq) + 'b')  # Convert integer to binary string
            
            register = binary_str[-nr:]
            
            if binary_str in results_me: 
                sum_dict[register] +=results_me[binary_str]
            
                if binary_str[0]=='1':
                            p_vec[int(register,2)] = results_me[binary_str]
     
        for i, (register, s) in enumerate(sum_dict.items()):
            if s != 0:
                p_vec[i] /= s
            else:                                    
                p_vec[i]=0.5              #safeguard in case register is not measured
    
        p_vec=p_vec.tolist() #results_me #if i want to double check the results
        return p_vec #,results_me
             
    rhobeg = 0.1*2*np.pi  # Radius of Cobyla linear approximation. Adjust this value based on your problem and experimentation if COBYLA struggles
                      # BUT make sure to check if you have enough shots and depth before touching this parameter
    
    def cfgrad_vec(theta, uppertriangularmatrix): # Calculates gradient of cost function
        # Returns 2 outputs
        
        # sampling the circuit at theta 
        proj_, proj_1 = proj_list(run_job(theta),na,nr) # P(theta), proj_1/proj is the probability of the ancillas
    
        probability = proj_1/proj_
        
        #initialize the gradient vector 
    
        grad_vec= np.zeros(depth*nq)
    
        #we look at the jacobian of the matrix so we need to evaluate the derivative of the cost function according to each theta_i
    
        for i in range(nq*depth):
    
        #we need to calculate the circuit at theta_1, ..., theta_i+pi/2, ..., theta_n
        #we need to calculate the circuit at theta_1, ..., theta_i-pi/2, ..., theta_n
        #according to the parameter shift rule : 
            
            pi=np.zeros(nq*depth)
            pi[i]=np.pi/2    
    
            
            pi=np.concatenate((np.zeros(nq),pi))   #account for the first layer that does not count
            
            theta_min=theta-pi
            theta_max=theta+pi
            
        #we extract the projectors list : 
            proj_min , proj_min_1 = proj_list(run_job(theta_min), na ,nr)
            proj_max , proj_max_1 = proj_list(run_job(theta_max), na ,nr)
        
    
            dproj_1 = 0.5*(proj_max_1 - proj_min_1) # Parameter shift for ancilla 
            dproj_ = 0.5*(proj_max - proj_min) # Parameter shift for registers
            
        
            # anc_d_prob = (dproj_1/proj_) - (proj_1/(proj_)**2)*dproj_
            #this is the vector of the different derivatives of sampling a variable
            #  dprob[k] gives us the derivative according to theta_i of the probability of sampling the k-th variable
            dprob = np.multiply(1/proj_, dproj_1) - np.multiply(np.multiply(np.multiply(1/proj_,1/proj_),proj_1), dproj_)
    
            
            #the derivative of the cost function according to theta_i is the sum over nc of the coefficient of the matrix Q and the derivative of 
            #the sampling probability that is to say : 
    
            #  dcf/dtheta_i = sum for k!=l of (Q_kl *(dprob[k]*prob[l] + dprob[l]*prob[k]) which are the off diagonal terms 
        
            #  and of k of (Q_k * dprob[k]) which are the diagonal terms 
        
            # here, for the off diagonal terms, we used the product rule of derivatives 
        
            #let's calculate the diagonal term first
            for k in range(nc): 
                if uppertriangularmatrix[k][0]!=0:
                    grad_vec[i]+=uppertriangularmatrix[k][0]*dprob[k]
    
            #let's calculate the off diagonal terms now : 
    
            #since the qubo matrix is symmetric, we can only compute the upper diagonal terms 
            for k in range(nc-1):
                for l in range(k+1,nc-k):
                    if uppertriangularmatrix[k][l]!=0:
                        grad_vec[i]+=uppertriangularmatrix[k][l]*(dprob[k]*probability[l] + dprob[l]*probability[k])
                
        return grad_vec

    def jac(theta):
        return cfgrad_vec(theta,Q_upper_triangular)

    def build_callback(callback_dict):
        """Return callback function and stores intermediate values into a dictionary.
    
            callback_dict (dict): Mutable dict for storing values"""
    
        def callback(current_vector):
            """Callback function storing previous solution vector,
            computing the intermediate cost value, and displaying number
            of completed iterations and average time per iteration.
    
            Values are stored in pre-defined 'callback_dict' dictionary.
    
            Parameters:
                current_vector (ndarray): Current vector of parameters
                                          returned by optimizer
            """
            # Keep track of the number of iterations
            callback_dict["iters"] += 1
            # Set the prev_vector to the latest one
            callback_dict["prev_vector"] = current_vector
            # Compute the value of the cost function at the current vector
            # This adds an additional function evaluation
            current_cost = cff_me_faster(current_vector)
            callback_dict["cost_history"].append(current_cost)
            callback_dict["jac"].append(jac(current_vector))
    
            # Print to screen on single line
            print(
                "Iters. done: {} [Current cost: {}]".format(callback_dict["iters"], current_cost),
                end="\r",
                flush=True,
            )

        return callback
    
    #List of sampling of classical solution 

    classical_solutions=[]
    
    ##creating callback dictionary
    
    callback_dict = {
        "prev_vector": None,
        "iters": 0,
        "cost_history": [],
        "jac" : []

    }
    callback = build_callback(callback_dict)

    if SGD == 'rmsprop':  
        def sgd(
            fun,
            x0,
            jac,
            args=(),
            learning_rate=0.2,
            gamma=0.9,
            eps=1e-8,
            startiter=0,
            maxiter=max_iter,
            callback=None,
            **kwargs
        ):
            """``scipy.optimize.minimize`` compatible implementation of root mean
            squared prop: See Adagrad paper for details.
            Adapted from ``autograd/misc/optimizers.py``.
            """
            x = x0
            avg_sq_grad = np.ones_like(x)
        
            for i in range(startiter, startiter + maxiter):
                g = np.concatenate((np.zeros(nq),jac(x)))
        
                if callback and callback(x):
                    break
        
                avg_sq_grad = avg_sq_grad * gamma + g**2 * (1 - gamma)
                x = x - learning_rate * g / (np.sqrt(avg_sq_grad) + eps)
        
            i += 1
            return OptimizeResult(x=x, fun=fun(x), jac=g, nit=i, nfev=i, success=True)


    elif SGD == 'sgd':
        def sgd(
            fun,
            x0,
            jac,
            args=(),
            learning_rate=0.1,
            mass=0.9,
            startiter=0,
            maxiter=30,
            callback=None,
            **kwargs
        ):
            """``scipy.optimize.minimize`` compatible implementation of stochastic
            gradient descent with momentum.
            Adapted from ``autograd/misc/optimizers.py``.
            """
            x = x0
            velocity = np.zeros_like(x)
        
            for i in range(startiter, startiter + maxiter):
                g = np.concatenate((np.zeros(nq),jac(x)))
        
                if callback and callback(x):
                    break
        
                velocity = mass * velocity - (1.0 - mass) * g
                x = x + learning_rate * velocity
        
            i += 1
            return OptimizeResult(x=x, fun=fun(x), jac=g, nit=i, nfev=i, success=True)

    elif SGD == 'adam':

        def adam(
            fun,
            x0,
            jac,
            args=(),
            learning_rate=0.01,
            beta1=0.9,
            beta2=0.999,
            eps=1e-10,
            startiter=0,
            maxiter=max_iter,
            callback=None,
            **kwargs
        ):
            """``scipy.optimize.minimize`` compatible implementation of ADAM -
            [http://arxiv.org/pdf/1412.6980.pdf].
            Adapted from ``autograd/misc/optimizers.py``.
            """
            x = x0
            m = np.zeros_like(x)
            v = np.zeros_like(x)
        
            for i in range(startiter, startiter + maxiter):
                g = np.concatenate((np.zeros(nq),jac(x)))
        
                if callback and callback(x):
                    break
        
                m = (1 - beta1) * g + beta1 * m  # first  moment estimate.
                v = (1 - beta2) * (g**2) + beta2 * v  # second moment estimate.
                mhat = m / (1 - beta1**(i + 1))  # bias correction.
                vhat = v / (1 - beta2**(i + 1))
                x = x - learning_rate * mhat / (np.sqrt(vhat) + eps)
        
            i += 1
            return OptimizeResult(x=x, fun=fun(x), jac=g, nit=i, nfev=i, success=True)
    
    # Execute the variational quantum circuit
    
    start_time = time.time()
    
    for _ in range(num_solutions):              # Execute quantum circuit and obtain outcomes
            
        result_rms = adam(cff_me_faster, [np.pi * np.random.uniform(0, 2) for i in range(nq*(depth + 1))],
                                      
                                      jac, 
                                      
                                      callback=callback)    
        
        classical_solutions.append(result_rms.x)
        
    end_time = time.time()
    
    execution_time = end_time - start_time
    
    jac_history = callback_dict['jac']
    # Extract the cost history from the dictionary
    cost_history_normalized = [(cost-C_min)/(C_max-C_min) for cost in callback_dict['cost_history']]
    #cost_history_normalized = [cost for cost in callback_dict['cost_history']]

    # Create figure and axis objects for two subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

        # Plot normalized cost history on the first subplot
    ax1.plot(range(len(cost_history_normalized)), cost_history_normalized, label='Normalized Cost', linewidth=2, color='blue')
    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('Normalized Cost', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')
    ax1.grid(True)
    ax1.legend()


        # Set y-axis range for normalized cost subplot
    ax1.set_ylim([0, 1])
    
    # Plot Jacobian on the second subplot
    ax2.plot(range(len(jac_history)), jac_history, label='Jacobian', linestyle='--')
    ax2.set_xlabel('Iteration')
    ax2.set_ylabel('Jacobian')
    ax2.tick_params(axis='y')
    ax2.grid(True)
    ax2.legend()
    
    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.show()

        # ANSI escape code for bold text
    bold_text = "\033[1m"
        # ANSI escape code to reset text formatting
    reset_format = "\033[0m"

    print("The solutions are : ", classical_solutions)
    #print("The normalised cost history is : ", cost_history_normalized)

    solution_average_me = 0

    shots = 30000 #to get a precise solution 

    # Print the text in bold
    for i in range(num_solutions):
        #for j in range(17): 
            #print(bold_text + "The solution is", format(np.round((sample10(sample_to_vec(final_result_cobyla_me_dp[i]))[j])))
        print(bold_text + "The solution is", format(np.round((prob_vec(classical_solutions[i]))))
         + reset_format)
        #print(bold_text + "The solution value is", format(np.dot(np.dot(np.transpose(np.round((sample10(sample_to_vec(final_result_cobyla_me_dp[i]))[j]))),Q),np.round((sample10(sample_to_vec(final_result_cobyla_me_dp[i]))[j]))))
        print(bold_text + "The solution value is", format(np.dot(np.dot(np.transpose(np.round((prob_vec(classical_solutions[i])))),Q),np.round((prob_vec(classical_solutions[i])))))
         + reset_format) 
        solution_average_me+=np.dot(np.dot(np.transpose(np.round((prob_vec(classical_solutions[i])))),Q),np.round((prob_vec(classical_solutions[i]))))
        print(f"""probability matrix value in percentage: {(np.floor(np.multiply(prob_vec(classical_solutions[i]),100)))}""")
    print(f"""Number of iterations per optimization : {callback_dict['iters']/num_solutions}""")
    print(f"""Number of second per iteration : {execution_time/callback_dict['iters'] } seconds""")
    print(f"""Number of second per optimization : {execution_time/num_solutions } seconds""")
    print(f"""Time (mins): {execution_time//60} mins and {execution_time%60} sec""")
    print(f"""Cost function value: {callback_dict['cost_history'][-1]}""")
    print(f"""Solution average: {solution_average_me/num_solutions}""")


### Solve your problem here using COBYLA

In [10]:
shots =  10000
depth = 5
noise = False
num_solutions = 1
max_iter = 300
optimization_precision = 1e-6

backend = 'ibm_sherbrooke' ## to change it you have to change the min_encoding code when you initialize the sampler

minimal_encoding(Q, shots, depth, noise, 1, num_solutions, max_iter, optimization_precision)

Iters. done: 300 [Current cost: -8.486978622529419]
   Return from subroutine COBYLA because the MAXFUN limit has been reached.

   NFVALS =  300   F =-8.445494E+00    MAXCV = 0.000000E+00
   X = 5.844471E+00   5.414491E+00   1.486083E+00   4.811926E+00   3.021590E+00
       5.777831E+00   5.925741E+00   3.404889E+00   6.283184E+00   4.983102E+00
       3.494708E-02   4.817773E+00   2.023827E+00   6.081268E+00   9.890335E-01
       6.278848E+00   4.674606E-01   1.751909E+00   4.928003E+00   3.768789E+00
       6.258665E+00   4.816332E-01   5.712643E+00   9.716736E-01
Iters. done: 301 [Current cost: -8.35861493213987]
The solutions are :  [array([5.8444707 , 5.4144908 , 1.48608259, 4.81192579, 3.02158999,
       5.77783086, 5.92574064, 3.40488867, 6.28318433, 4.98310224,
       0.03494708, 4.81777329, 2.02382687, 6.08126806, 0.98903354,
       6.27884788, 0.46746059, 1.75190939, 4.92800336, 3.76878947,
       6.25866514, 0.48163323, 5.71264323, 0.97167362])]

The solution is [1. 1. 0. 1

### Solve your problem here  using SGD

In [11]:
shots =  7000
depth = 4
noise = False
num_solutions = 1
max_iter = 10
optimization_precision = 1e-6

backend = 'ibm_sherbrooke' ## to change the backend, you have to change the min_encoding code when you initialize the sampler

minimal_encoding_sgd(Q, shots, depth, noise, 1, num_solutions, 'adam')

The solutions are :  [array([0.38863549, 1.01711223, 4.50455223, 2.7340925 , 4.30765979,
       5.17692629, 2.55549792, 4.16194901, 5.46313443, 1.67951457,
       2.30613418, 5.21592069, 5.64877226, 5.86689286, 3.78339812,
       0.89622081, 1.3771796 , 6.25179569, 1.90201104, 4.25169451])]
The solution is [1. 1. 0. 0. 1. 0. 1. 1.]
The solution value is 14.0
probability matrix value in percentage: [53. 81.  6.  4. 99. 11. 99. 88.]
Number of iterations per optimization : 10.0
Number of second per iteration : 3.2801820993423463 seconds
Number of second per optimization : 32.80182099342346 seconds
Time (mins): 0.0 mins and 32.80182099342346 sec
Cost function value: 14.147045852915682
Solution average: 14.0


## Angel Q algorithm : selective encoding

#### Selective encoding solver

In [96]:
def selective_encoding(na, nr, Q, shots, depth, noise, optimization_level, num_solutions, max_iter, tol):
    import time

    nc = len(Q)
    
    nr=int(math.log2(nc/na))
    
    nq=nr+na

    Q_upper_triangular=uppertriangularmatrix(Q)

    
     # ANSI escape code for bold text
    bold_text = "\033[1m"
        # ANSI escape code to reset text formatting
    reset_format = "\033[0m"

    print( bold_text + f"# ancilla qubits : {na}, # register qubits : {nr} , NUMBER OF QUBITS :{nq}" + reset_format)
    print("")
    #create ansatz

    #initialize the quantum circuit
    c_se=QuantumCircuit(nq)
    for i in range(nq):
         c_se.h(i)

    theta_se=[0 for i in range(nq)] +[np.pi for i in range(nq*(depth))]
    
    
    #to cancel first rotation
    for i in range(nq):
        c_se.ry(-theta_se[i],i)
    
    #parameters
    ansatz_se= RealAmplitudes(nq, entanglement='pairwise',reps=depth,parameter_prefix='theta_se', insert_barriers=True)
    
    c_se.compose(ansatz_se, inplace=True)
    c_se.measure_all()
    #print(c_se,theta_se)

    
    """""Sampling routine"""""

    #initialization of the sampler
    if noise == False : 
        sampler = StatevectorSampler()
        isa_c_se= c_se
    else : 
        service = QiskitRuntimeService()
 
        # Specify a system to use for the noise model
        real_backend = service.backend("ibm_sherbrooke")
        aer = AerSimulator.from_backend(real_backend)
        
        # Run the sampler job locally using AerSimulator.
        pm = generate_preset_pass_manager(backend=aer, optimization_level=optimization_level)  ## 

        isa_c_se = pm.run(c_se)
        #qc_noise2.draw("mpl"), isa_qc2.draw("mpl")
        sampler = SamplerV2(backend=aer)

    def run_job(theta_se):
        job = sampler.run([(isa_c_se, theta_se)],shots=shots)
        job_result = job.result()
        return job_result
    
    
    #create probability vector and samples at the same time
    def prob_vec_se(theta):
        
        #anc_proj = np.zeros((na*nr))  
        anc_proj = [np.zeros(na) for i in range(2**nr)]
    
        job_result_se = run_job(theta)
        
        #sorted dictionary
        results_se={k: job_result_se[0].data.meas.get_counts()[k] for k in sorted(job_result_se[0].data.meas.get_counts())}
    
        reg_dict = {format(k, '0' + str(nr) + 'b') : 0 for k in range(2**nr)}
    
        for i in range(2**nq):          ## to explore the whole range of solution samples
            
            binary_str = format(i, '0' + str(nq) + 'b')  # Convert integer to binary string
            
            if binary_str in results_se: 
    
                register = binary_str[-nr:] #takes the register qubit values of the solution
    
                ancilla = binary_str[:na]   #takes the ancilla qubit values of the solution
                
                reg_dict[register] +=results_se[binary_str]   #update the register counts
            
                for k in range(len(ancilla)):  ##based on the ancilla values, update the counts of x_i = 1, for x_i in register measured
                                 
                    if ancilla[k]=='1': 
                        anc_proj[int(register,2)][k] += results_se[binary_str]
                    
        for i, (register, s) in enumerate(reg_dict.items()):
            if s != 0:
                anc_proj[int(register,2)] /= s
            else:                                    
                anc_proj[int(register,2)]=[0.5]*na          #safeguard in case register is not measured
    
        #anc_proj=anc_proj #results_me #if i want to double check the results
        def sample_to_vec_se(vector): 
            l=[]
            for reg in range(2**nr): 
                a = vector[reg]
                for i in range(na):
                    l.append(a[i])
            return l
    
        return sample_to_vec_se(anc_proj)#,results_se, reg_dict
        

    """""Cost function calculations"""""


    ## cost function computation 

    def faster(p_vector,upper_triangular_qubo_matrix):
        ## first, we construct the C matrix 
        C= [[0 for j in range(i, nc)] for i in range(nc)]
        # Step 1: Calculate the diagonal coefficients of C
            
        # Iterate over the indices of matrix qubo_matrix
        for i in range(nc):
            # Check if the element is on the diagonal and non-zero
                # Assign 1 to diagonal elements of C
            C[i][0] = p_vector[i]
        
        # Step 2: Compute the non-diagonal coefficients of C
        # Iterate over the indices of matrix qubo_matrix
        for i in range(nc):
            for j in range(1, nc-i):
                # Check if the corresponding diagonal element of A is non-zero
                if upper_triangular_qubo_matrix[i][j] != 0:
                    # Assign to non-diagonal elements of C
                    C[i][j] = C[i][0]*C[j+i][0] 
        return C
    
    def cf_se_faster(qubo_matrix, theta):
        S=0
        A=faster(prob_vec_se(theta),Q_upper_triangular)
        for i in range(nc): 
            for j in range(0, nc-i):
                S+=qubo_matrix[i][j]*A[i][j]
        return S
    
    def cff_se_faster(theta):
            return cf_se_faster(Q_upper_triangular,theta)



    """""Optimisation routine"""""



    rhobeg = 0.1*2*np.pi  # Adjust this value based on your problem and experimentation


    def build_callback_se(callback_dict_se):
    
        def callback_se(current_vector):
  
            # Keep track of the number of iterations
            callback_dict_se["iters"] += 1
            
            # Set the prev_vector to the latest one
            callback_dict_se["prev_vector"] = current_vector
            
            # Compute the value of the cost function at the current vector
            # This adds an additional function evaluation
            current_cost = cff_se_faster(current_vector)
            callback_dict_se["cost_history"].append(current_cost)
            
            # Print to screen on single line
            print(
                "Iters. done: {} [Current cost: {}]".format(callback_dict_se["iters"], current_cost),
                end="\r",
                flush=True,
            )
    
        return callback_se

    #List of sampling of classical solution 
    
    classical_solutions_se=[]
    
    ##creating callback dictionary
    
    callback_dict_se = {
        "prev_vector": None,
        "iters": 0,
        "cost_history": [],
    }
    callback_se = build_callback_se(callback_dict_se)
    
    # Execute the variational quantum circuit
    start_time = time.time()
    
    for _ in range(num_solutions):              # Execute quantum circuit and obtain outcomes
    
            
        result_cobyla_se = minimize(cff_se_faster, [np.pi * np.random.uniform(0, 2) for i in range(nq*(depth + 1))],
                                   
                                       method="COBYLA",
                                   
                                       bounds=[(0, 2*np.pi) for _ in range(nq*(depth + 1))],
                                   
                                       callback=callback_se,
                                   
                                       options={'maxiter': max_iter, 'tol': tol, 'disp': True})
        
        classical_solutions_se.append(result_cobyla_se.x)
        
    end_time = time.time()
    #result_cobyla_me_dp
    
    execution_time = end_time - start_time


    # Extract the cost history from the dictionary
    cost_history_normalized_se = [(cost-C_min)/(C_max-C_min) for cost in callback_dict_se['cost_history']]
    
    # Plot the cost history
    plt.plot(cost_history_normalized_se)
    plt.xlabel('Iteration')
    plt.ylabel('Cost')
    plt.title(f'Cost History Normalized,  nc = {nc},   depth = {depth},   shots = {shots},   noise = {noise},   iter max = {max_iter}')
    plt.grid(True)
    plt.show()


         # ANSI escape code for bold text
    bold_text = "\033[1m"
        # ANSI escape code to reset text formatting
    reset_format = "\033[0m"
    print("")
    print("The solutions are : ", classical_solutions_se)
    #print("The normalised cost history is : ", cost_history_normalized)
    print("")

    solution_average_se = 0

    shots = 30000 #to get a precise solution 

    # Print the text in bold
    for i in range(num_solutions):
        #for j in range(17): 
            #print(bold_text + "The solution is", format(np.round((sample10(sample_to_vec(final_result_cobyla_me_dp[i]))[j])))
        print(bold_text + "The solution is", format(np.round((prob_vec_se(classical_solutions_se[i]))))
         + reset_format)
        #print(bold_text + "The solution value is", format(np.dot(np.dot(np.transpose(np.round((sample10(sample_to_vec(final_result_cobyla_me_dp[i]))[j]))),Q),np.round((sample10(sample_to_vec(final_result_cobyla_me_dp[i]))[j]))))
        print(bold_text + "The solution value is", format(np.dot(np.dot(np.transpose(np.round((prob_vec_se(classical_solutions_se[i])))),Q),np.round((prob_vec_se(classical_solutions_se[i])))))
         + reset_format) 
        print("")

        solution_average_se+=np.dot(np.dot(np.transpose(np.round((prob_vec_se(classical_solutions_se[i])))),Q),np.round((prob_vec_se(classical_solutions_se[i]))))
        print(f"""probability matrix value in percentage: {(np.floor(np.multiply(prob_vec_se(classical_solutions_se[i]),100)))}""")
    print(f"""Number of iterations per optimization : {callback_dict_se['iters']/num_solutions}""")
    print(f"""Number of second per iteration : {execution_time/callback_dict_se['iters'] } seconds""")
    print(f"""Number of second per optimization : {execution_time/num_solutions } seconds""")
    print(f"""Time (mins): {execution_time//60} mins and {execution_time%60} sec""")
    print(f"""Cost function value: {callback_dict_se['cost_history'][-1]}""")
    print(f"""Solution average: {solution_average_se/num_solutions}""")


### solve your problem here using COBYLA

In [103]:
shots =  7000
depth = 6
noise = False
num_solutions = 1
max_iter = 150
optimization_precision = 1e-6

na = 4
nr = 1

backend = 'ibm_sherbrooke' ## to change it you have to change the min_encoding code when you initialize the sampler

selective_encoding(na, nr, Q, shots, depth, noise, 1, num_solutions, max_iter, optimization_precision)

# ancilla qubits : 4, # register qubits : 1 , NUMBER OF QUBITS :5

Iters. done: 150 [Current cost: -20.807624070136043]
   Return from subroutine COBYLA because the MAXFUN limit has been reached.

   NFVALS =  150   F =-2.107792E+01    MAXCV = 2.976225E-16
   X = 5.634104E+00   3.473794E+00   3.792682E+00   2.480701E+00   5.593363E+00
       1.122574E+00   3.379489E+00  -2.976225E-16   1.889587E+00   1.464609E+00
       3.826675E-01   5.321020E+00   8.579498E-01   4.914279E-01   1.729868E+00
       6.283185E+00   6.261954E+00   5.246363E+00   2.924964E+00   6.283185E+00
       2.992950E+00   1.038347E+00   1.741030E+00   5.655223E+00   3.102974E-01
       5.114702E+00   1.586744E+00   4.209487E+00   2.503495E+00   6.281965E+00
       1.877232E+00   1.765800E+00   8.240603E-01   2.821814E+00   4.577888E-01
Iters. done: 151 [Current cost: -21.09623964488295]
The solutions are :  [array([ 5.63410450e+00,  3.47379382e+00,  3.79268223e+00,  2.48070077e+00,
        5.59336349e+00,  1.1225740